In [21]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from collections import OrderedDict

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [36]:
#loading the data
train=pd.read_csv('C:/Users/Srinivas/Downloads/Desktop/train.csv')

In [37]:
#loading the data
test=pd.read_csv('C:/Users/Srinivas/Downloads/Desktop//test.csv')

In [38]:
#dropping few columns
train.drop(["homepage","imdb_id","belongs_to_collection","tagline","status","original_title","overview","genres","poster_path","production_countries", "production_companies","release_date","spoken_languages","title",'Keywords', 'cast', 'crew'], axis = 1,inplace=True)
test.drop(["homepage","imdb_id","belongs_to_collection","tagline","status","original_title","overview","genres","poster_path","production_countries", "production_companies","release_date","spoken_languages","title",'Keywords','cast', 'crew'], axis =1,inplace = True)

In [39]:
from sklearn.preprocessing import LabelEncoder
#label encode original language

le = LabelEncoder()
train['original_language'] = le.fit_transform(train['original_language'])
train.head(1)

,id,budget,original_language,popularity,runtime,revenue
0,1,14000000,7,6.575393,93.0,12314651


In [54]:
test['original_language'] = le.fit_transform(test['original_language'])
test.head()

,id,budget,original_language,popularity,runtime
0,3001,0,19,3.851534,90.0
1,3002,88000,9,3.559789,65.0
2,3003,0,9,8.085194,100.0
3,3004,6800000,12,8.596012,130.0
4,3005,2000000,9,3.217680,92.0


In [41]:
#Imputing missing values for both train and test
train.fillna(train['runtime'].mode() ,inplace=True)
test.fillna(test['runtime'].mode()  ,inplace=True)


In [55]:
df=pd.DataFrame()
df=pd.concat([test,train],sort=False) 

In [56]:
df.tail()

,id,budget,original_language,popularity,runtime,revenue
2995,2996,0,7,9.853270,102.0,1596687.0
2996,2997,0,29,3.727996,102.0,180590.0
2997,2998,65000000,7,14.482345,120.0,89456761.0
2998,2999,42000000,7,15.725542,90.0,171963386.0
2999,3000,35000000,7,10.512109,106.0,82087155.0


In [57]:
df.fillna(df['revenue'].mean(),inplace=True)

In [58]:
df.head()

,id,budget,original_language,popularity,runtime,revenue
0,3001,0,19,3.851534,90.0,6.672585e+07
1,3002,88000,9,3.559789,65.0,6.672585e+07
2,3003,0,9,8.085194,100.0,6.672585e+07
3,3004,6800000,12,8.596012,130.0,6.672585e+07
4,3005,2000000,9,3.217680,92.0,6.672585e+07


In [59]:
df.tail()

,id,budget,original_language,popularity,runtime,revenue
2995,2996,0,7,9.853270,102.0,1596687.0
2996,2997,0,29,3.727996,102.0,180590.0
2997,2998,65000000,7,14.482345,120.0,89456761.0
2998,2999,42000000,7,15.725542,90.0,171963386.0
2999,3000,35000000,7,10.512109,106.0,82087155.0


In [60]:
X = df.drop(['revenue'],axis=1)
y = df.revenue

from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.7, random_state=500)

In [61]:
import xgboost
from xgboost import XGBRegressor

In [62]:
# fit model no training data
model = XGBRegressor()
model.fit(X_train, y_train)

C:\Users\Srinivas\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [63]:
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer()
imputed_X_train = my_imputer.fit_transform(X_train)
imputed_X_test = my_imputer.transform(test)

In [64]:
from sklearn.ensemble import GradientBoostingRegressor

In [65]:
clf = GradientBoostingRegressor(n_estimators=100, learning_rate=1.0, max_depth=1)
clf.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=1.0, loss='ls', max_depth=1, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [84]:
pr_test=model.predict(test)

In [83]:
pr_train=model.predict(X_train)

In [68]:
pr_test

array([50465400., 50465400., 61488744., ..., 67191704., 75429610.,
       63161348.], dtype=float32)

In [85]:
pr_train

array([62562628., 76891700., 65598344., ..., 66879210., 67184904.,
       27077972.], dtype=float32)

In [74]:
values = pd.DataFrame({'id': test.id , 'revenue': pr_test})


In [75]:
values.tail()

,id,revenue
4393,7394,71849184.0
4394,7395,65131148.0
4395,7396,67191704.0
4396,7397,75429608.0
4397,7398,63161348.0


In [76]:
values.head()

,id,revenue
0,3001,50465400.0
1,3002,50465400.0
2,3003,61488744.0
3,3004,65035220.0
4,3005,50465400.0


In [77]:
values[3401:3600]

,id,revenue
3401,6402,61488744.0
3402,6403,66841568.0
3403,6404,67603992.0
3404,6405,65328212.0
3405,6406,65153472.0
3406,6407,62853884.0
3407,6408,65215800.0
3408,6409,70493440.0
3409,6410,62063588.0
3410,6411,65598344.0


In [107]:
values.to_csv("C:/Users/Srinivas/AppData/Roaming/jupyter/runtime/values.csv",header=True,index=False)

In [80]:
values.shape

(4398, 2)

In [93]:
from sklearn import metrics
RMSE_train = np.sqrt( metrics.mean_squared_error(y_train, pr_train))

#print('RMSE for training set is {}'.format(RMSE_train))
yhat = model.predict(X_train)
SS_Residual = sum((y_train-yhat)**2)
SS_Total = sum((y_train-np.mean(y_train))**2)
r_squared = 1 - (float(SS_Residual))/SS_Total
adjusted_r_squared = 1 - (1-r_squared)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
print("r_squared for train data ",r_squared, " and adjusted_r_squared for train data",adjusted_r_squared)


r_squared for train data  0.8772834250572499  and adjusted_r_squared for train data 0.8770061621224493


we got r-squared value 87% from this we,can say that the model performance is 87%

In [94]:
values.revenue.sum()

292958370000.0

In [106]:
train.revenue.sum()

200177555666

we can see that the predicted revenue is some higher compared with the current and previous revenues..... 